In [40]:
import numpy as np
import pandas as pd 
from random import random
import time
import warnings
import shap
import os 

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import statsmodels.api as sm
from sklearn import metrics
from torchmetrics import MeanSquaredError, MeanAbsoluteError, R2Score

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

warnings.filterwarnings(action='ignore') 

# 1.Make data

In [62]:
# Simple preprocessing and standardization of train data
le = LabelEncoder()

data = pd.read_csv('./data/train.csv') # Replacing Missing values using k-means
X_data = data.drop(['freq'],axis = 1)
y_data = data['freq']

# Used to encode stnNm features
le.fit(X_data['stnNm'])
X_data['stnNm'] = le.transform(X_data['stnNm'])

# Converting to an integer type of feature related to the population 
for col in X_data.columns:
    if X_data[col].dtype == 'object':
        X_data[col] = X_data[col].apply(lambda x : x.replace(',',''))
        X_data[col] = X_data[col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, test_size=0.2, random_state=328)

# 2. VIF

In [22]:
def check_vif(dataframe):
    dataframe = add_constant(dataframe)
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(
        dataframe.values, i) for i in range(dataframe.shape[1])]
    vif["features"] = dataframe.columns
    return vif

In [23]:
remove_list = []
t1 = time.time()
X_train_vif = X_train.copy()

for _ in range(1000):
    
    vif = check_vif(X_train_vif)

    if len(vif[vif['VIF Factor'] >=10]) >=1:
        vif_value = round(vif[vif['VIF Factor'] >=10].sort_values('VIF Factor', ascending=False).iloc[0,0],3)
        remove_col = vif[vif['VIF Factor'] >=10].sort_values('VIF Factor', ascending=False).iloc[0,1]

        if (remove_col == 'const') & (vif[vif['VIF Factor'] >=10].shape[0] == 1):
            print('VIF가 10이 넘는 변수가 없습니다. === FOR LOOP을 종료합니다.')
            break

        elif (remove_col == 'const') & (vif[vif['VIF Factor'] >=10].shape[0] != 1):
            vif_value = round(vif[vif['VIF Factor'] >=10].sort_values('VIF Factor', ascending=False).iloc[1,0],3)
            remove_col = vif[vif['VIF Factor'] >=10].sort_values('VIF Factor', ascending=False).iloc[1,1]
            remove_list.append(remove_col)
            X_train_vif.drop(remove_col, axis=1, inplace=True)
            t2 = time.time()
            elapsed_time = t2-t1
            print('VIF값이 '+str(vif_value)+'인 '+remove_col+'이 제거되었습니다. === 현재 총 제거된 변수의 개수는 '+str(len(remove_list))+'개 입니다. === 경과된 시간: '+str(round(elapsed_time/60))+'분')

        else:
            remove_list.append(remove_col)
            X_train_vif.drop(remove_col, axis=1, inplace=True)
            t2 = time.time()
            elapsed_time = t2-t1
            print('VIF값이 '+str(vif_value)+'인 '+remove_col+'이 제거되었습니다. === 현재 총 제거된 변수의 개수는 '+str(len(remove_list))+'개 입니다. === 경과된 시간: '+str(round(elapsed_time/60))+'분')


    else:
        print('VIF가 10이 넘는 변수가 없습니다. === FOR LOOP을 종료합니다.')
        break

vif_col = X_train_vif.columns.to_list()
test_vif = X_test[vif_col]

print(vif_col)

VIF값이 inf인 10대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 1개 입니다. === 경과된 시간: 0분
VIF값이 40627.859인 총_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 2개 입니다. === 경과된 시간: 0분
VIF값이 1943.813인 40대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 3개 입니다. === 경과된 시간: 0분
VIF값이 1860.581인 avgTa이 제거되었습니다. === 현재 총 제거된 변수의 개수는 4개 입니다. === 경과된 시간: 0분
VIF값이 1387.397인 20대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 5개 입니다. === 경과된 시간: 0분
VIF값이 1322.82인 예보_3시간기온이 제거되었습니다. === 현재 총 제거된 변수의 개수는 6개 입니다. === 경과된 시간: 0분
VIF값이 650.819인 60대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 7개 입니다. === 경과된 시간: 0분
VIF값이 427.042인 90대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 8개 입니다. === 경과된 시간: 0분
VIF값이 411.085인 체감온도이 제거되었습니다. === 현재 총 제거된 변수의 개수는 9개 입니다. === 경과된 시간: 0분
VIF값이 275.13인 avgTd이 제거되었습니다. === 현재 총 제거된 변수의 개수는 10개 입니다. === 경과된 시간: 0분
VIF값이 175.203인 30대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 11개 입니다. === 경과된 시간: 0분
VIF값이 104.267인 예보_일최저기온이 제거되었습니다. === 현재 총 제거된 변수의 개수는 12개 입니다. === 경과된 시간: 0분
VIF값이 97.416인 70대_인구수이 제거되었습니다. === 현재 총 제거된 변수의 개수는 13개 입니다. === 경과된 시간: 0분
VIF

# 3.Modeling

- 3-1. LGBM
- 3-2. Random Forest
- 3-3. XGBoost
- 3-4. Stacking 


In [30]:
def lgb_objective(trial):
    lgb_learning_rate = trial.suggest_float('learning_rate', 0.04, 0.4)
    lgb_leaves= trial.suggest_int('num_leaves', 10, 1000)
    lgb_bytree = trial.suggest_float("colsample_bytree", 0.1,0.3)
    lgb_subsample = trial.suggest_float("subsample", 0.1,0.3)
    lgb_depth =  trial.suggest_int('max_depth', 3, 100)
    lgb_child_samples = trial.suggest_int('min_child_samples', 3, 2000)
    lgb_alpha =  trial.suggest_loguniform('reg_alpha', 1e-8, 10.0)
    lgb_lambda = trial.suggest_loguniform('reg_lambda', 1e-8, 1.0)
    lgb_smooth = trial.suggest_int('cat_smooth', 1, 100)
    lgb_gain_to_split = trial.suggest_float('min_split_gain', 0.0, 30.0)
    lgb_max_bin = trial.suggest_int('max_bin',2,100)
    lgb_boosting = trial.suggest_categorical('boosting_type', ['gbdt','dart'])
    lgb_bagging = trial.suggest_uniform('bagging_fraction', 0.1, 1.0)
    lgb_bagging_freq =  trial.suggest_int('bagging_freq', 0, 15)

    regressor_obj = LGBMRegressor(boosting_type=lgb_boosting,objective='regression', metric='rmse', verbosity = -1,num_leaves=lgb_leaves,learning_rate=lgb_learning_rate
                                ,colsample_bytree= lgb_bytree, subsample=lgb_subsample, max_depth=lgb_depth, min_child_samples=lgb_child_samples,reg_alpha=lgb_alpha,
                                reg_lambda=lgb_lambda,cat_smooth=lgb_smooth,min_split_gain=lgb_gain_to_split,max_bin = lgb_max_bin
                                ,bagging_fraction=lgb_bagging,bagging_freq=lgb_bagging_freq)
                              

    rmse = np.sqrt(-cross_val_score(regressor_obj, X_train, y_train, scoring="neg_mean_squared_error", cv = 12, n_jobs=8))
    rmse =  np.mean(rmse)
    return rmse

def LGBMregreesr(trials):
    sampler = TPESampler(seed=42) # TPESampler --> MAE가 최소가 되는 방향으로 학습 진행 (MAE: 평균절대오차!)

    study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
    study_lgb.optimize(lgb_objective, n_trials=trials)

    print("Number of finished trials: {}".format(len(study_lgb.trials)))

    print("Best trial:")
    trial = study_lgb.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return study_lgb

## 4-1. all data

In [37]:
# 전체 데이터 셋에 대해 최적의 파라미터 찾고 RFE를 통해 feature select 

model = LGBMregreesr(150)
lgb_best_all = model.best_params

model_lgb = LGBMRegressor(**lgb_best_all)

model_lgb.fit(X_train,y_train)
pred_lgb_all = model_lgb.predict(X_test)

[I 2022-11-24 13:40:19,716] A new study created in memory with name: no-name-395a270d-d25c-49fc-9603-a00680f8ec56
[I 2022-11-24 13:40:20,734] Trial 0 finished with value: 1.3108217297412192 and parameters: {'learning_rate': 0.17483444278505053, 'num_leaves': 952, 'colsample_bytree': 0.24639878836228102, 'subsample': 0.2197316968394073, 'max_depth': 18, 'min_child_samples': 314, 'reg_alpha': 3.3323645788192616e-08, 'reg_lambda': 0.08499808989182997, 'cat_smooth': 61, 'min_split_gain': 21.242177333881365, 'max_bin': 4, 'boosting_type': 'gbdt', 'bagging_fraction': 0.29110519961044856, 'bagging_freq': 2}. Best is trial 0 with value: 1.3108217297412192.
[I 2022-11-24 13:40:22,418] Trial 1 finished with value: 1.2648880993969742 and parameters: {'learning_rate': 0.10602562354723619, 'num_leaves': 311, 'colsample_bytree': 0.20495128632644755, 'subsample': 0.18638900372842315, 'max_depth': 31, 'min_child_samples': 1225, 'reg_alpha': 1.8007140198129195e-07, 'reg_lambda': 2.1734877073417355e-06,

Number of finished trials: 150
Best trial:
  Value: 1.2503681000937097
  Params: 
    learning_rate: 0.11302067631051534
    num_leaves: 607
    colsample_bytree: 0.16791499158956036
    subsample: 0.2902545530796146
    max_depth: 10
    min_child_samples: 1617
    reg_alpha: 0.0011037311182397476
    reg_lambda: 0.18511224419945416
    cat_smooth: 85
    min_split_gain: 2.196979897839018
    max_bin: 92
    boosting_type: gbdt
    bagging_fraction: 0.9589013882249259
    bagging_freq: 5


## 4-2. RFE data

In [38]:
############################--------Make RFE DATASET--------#####################################
cv = KFold(n_splits=10, shuffle=True)
rfe = RFECV(estimator=LGBMRegressor(**lgb_best_all))

rfe.fit(X_train,y_train)

select_feature = []

for i,col in enumerate(X_train.columns):
    print('Column: %s, Selected: %s, Rank: %.3f' % (col, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True :
        select_feature.append(col)

for col in ['stnNm', 'sex']:
    if col not in select_feature:
        select_feature.append(col)

X_train_all_rfe = X_train[select_feature]
test_all_rfe = X_test[select_feature]

print(select_feature)

X_train = X_train_all_rfe

model = LGBMregreesr(150)
lgb_best_all_rfe = model.best_params

model_lgb_rfe = LGBMRegressor(**lgb_best_all_rfe)

model_lgb_rfe.fit(X_train,y_train)
pred_lgb_rfe = model_lgb_rfe.predict(test_all_rfe)

[LightGBM] [Warning] bagging_fraction is set=0.9589013882249259, subsample=0.2902545530796146 will be ignored. Current value: bagging_fraction=0.9589013882249259
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.9589013882249259, subsample=0.2902545530796146 will be ignored. Current value: bagging_fraction=0.9589013882249259
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.9589013882249259, subsample=0.2902545530796146 will be ignored. Current value: bagging_fraction=0.9589013882249259
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.9589013882249259, subsample=0.2902545530796146 will be ignored. Current value: bagging_fraction=0.9589013882249259
[LightGBM] [Warning] bagging

[I 2022-11-24 13:44:11,493] A new study created in memory with name: no-name-ebcec26b-d6cb-4655-971e-d678be1f36e5


Column: stnNm, Selected: True, Rank: 1.000
Column: tm, Selected: True, Rank: 1.000
Column: sex, Selected: False, Rank: 7.000
Column: avgTa, Selected: False, Rank: 8.000
Column: minTa, Selected: True, Rank: 1.000
Column: maxTa, Selected: False, Rank: 9.000
Column: sumRn, Selected: False, Rank: 4.000
Column: maxWs, Selected: True, Rank: 1.000
Column: avgWs, Selected: True, Rank: 1.000
Column: avgTd, Selected: False, Rank: 2.000
Column: minRhm, Selected: True, Rank: 1.000
Column: avgRhm, Selected: True, Rank: 1.000
Column: avgPv, Selected: False, Rank: 6.000
Column: avgPa, Selected: True, Rank: 1.000
Column: avgPs, Selected: True, Rank: 1.000
Column: avgTs, Selected: False, Rank: 3.000
Column: minTg, Selected: False, Rank: 5.000
Column: 체감온도, Selected: False, Rank: 10.000
Column: 총_인구수, Selected: True, Rank: 1.000
Column: 10세이하_인구수, Selected: True, Rank: 1.000
Column: 10대_인구수, Selected: True, Rank: 1.000
Column: 20대_인구수, Selected: True, Rank: 1.000
Column: 30대_인구수, Selected: True, Rank: 1

[I 2022-11-24 13:44:12,173] Trial 0 finished with value: 1.3183409290374077 and parameters: {'learning_rate': 0.17483444278505053, 'num_leaves': 952, 'colsample_bytree': 0.24639878836228102, 'subsample': 0.2197316968394073, 'max_depth': 18, 'min_child_samples': 314, 'reg_alpha': 3.3323645788192616e-08, 'reg_lambda': 0.08499808989182997, 'cat_smooth': 61, 'min_split_gain': 21.242177333881365, 'max_bin': 4, 'boosting_type': 'gbdt', 'bagging_fraction': 0.29110519961044856, 'bagging_freq': 2}. Best is trial 0 with value: 1.3183409290374077.
[I 2022-11-24 13:44:13,552] Trial 1 finished with value: 1.2647964156158669 and parameters: {'learning_rate': 0.10602562354723619, 'num_leaves': 311, 'colsample_bytree': 0.20495128632644755, 'subsample': 0.18638900372842315, 'max_depth': 31, 'min_child_samples': 1225, 'reg_alpha': 1.8007140198129195e-07, 'reg_lambda': 2.1734877073417355e-06, 'cat_smooth': 37, 'min_split_gain': 13.682099526511077, 'max_bin': 79, 'boosting_type': 'dart', 'bagging_fraction

Number of finished trials: 150
Best trial:
  Value: 1.250832370322102
  Params: 
    learning_rate: 0.06102179223357146
    num_leaves: 267
    colsample_bytree: 0.18064827959304697
    subsample: 0.11214569652425553
    max_depth: 13
    min_child_samples: 961
    reg_alpha: 0.010835038697755013
    reg_lambda: 0.015169703321577122
    cat_smooth: 100
    min_split_gain: 0.7712074264920915
    max_bin: 37
    boosting_type: gbdt
    bagging_fraction: 0.9640506266463021
    bagging_freq: 12


## 4-3. VIF data

In [28]:
X_train = X_train_vif

model = LGBMregreesr(150)
lgb_best_vif = model.best_params

model_lgb_vif = LGBMRegressor(**lgb_best_vif)

model_lgb_vif.fit(X_train,y_train)
pred_lgb_vif = model_lgb_vif.predict(test_vif)

[I 2022-11-24 13:17:37,101] A new study created in memory with name: no-name-5fef9af2-586a-4555-bc41-4e40f84200dd
[I 2022-11-24 13:17:37,659] Trial 0 finished with value: 1.379232307006334 and parameters: {'learning_rate': 0.17483444278505053, 'num_leaves': 952, 'colsample_bytree': 0.24639878836228102, 'subsample': 0.2197316968394073, 'max_depth': 18, 'min_child_samples': 314, 'reg_alpha': 3.3323645788192616e-08, 'reg_lambda': 0.08499808989182997, 'cat_smooth': 61, 'min_split_gain': 21.242177333881365, 'max_bin': 4, 'boosting_type': 'gbdt', 'bagging_fraction': 0.29110519961044856, 'bagging_freq': 2}. Best is trial 0 with value: 1.379232307006334.
[I 2022-11-24 13:17:38,959] Trial 1 finished with value: 1.3023601576084227 and parameters: {'learning_rate': 0.10602562354723619, 'num_leaves': 311, 'colsample_bytree': 0.20495128632644755, 'subsample': 0.18638900372842315, 'max_depth': 31, 'min_child_samples': 1225, 'reg_alpha': 1.8007140198129195e-07, 'reg_lambda': 2.1734877073417355e-06, '

Number of finished trials: 150
Best trial:
  Value: 1.2532958965650123
  Params: 
    learning_rate: 0.10692535094581457
    num_leaves: 225
    colsample_bytree: 0.24404581482588736
    subsample: 0.11888530757754323
    max_depth: 62
    min_child_samples: 1717
    reg_alpha: 0.06261861401562074
    reg_lambda: 0.4794437085751947
    cat_smooth: 45
    min_split_gain: 0.02014376412998231
    max_bin: 51
    boosting_type: gbdt
    bagging_fraction: 0.22401411479663555
    bagging_freq: 0
[LightGBM] [Warning] bagging_fraction is set=0.22401411479663555, subsample=0.11888530757754323 will be ignored. Current value: bagging_fraction=0.22401411479663555
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


# 5. Metrics & SHAP

In [58]:
import torch

MSE = MeanSquaredError()
MAE = MeanAbsoluteError()
R2 = R2Score()

print(f'MSE --> all_data : {MSE(torch.tensor(pred_lgb_all),torch.tensor(y_test.values))}, vif_data: {MSE(torch.tensor(pred_lgb_vif),torch.tensor(y_test.values))}, rfe_data : {MSE(torch.tensor(pred_lgb_rfe),torch.tensor(y_test.values))}')
print(f'MAE --> all_data : {MAE(torch.tensor(pred_lgb_all),torch.tensor(y_test.values))}, vif_data: {MAE(torch.tensor(pred_lgb_vif),torch.tensor(y_test.values))}, rfe_data : {MAE(torch.tensor(pred_lgb_rfe),torch.tensor(y_test.values))}')
print(f'R2_Score --> all_data : {R2(torch.tensor(pred_lgb_all),torch.tensor(y_test.values))}, vif_data: {R2(torch.tensor(pred_lgb_vif),torch.tensor(y_test.values))}, rfe_data : {R2(torch.tensor(pred_lgb_rfe),torch.tensor(y_test.values))}')

MSE --> all_data : 1.5797282457351685, vif_data: 1.5896488428115845, rfe_data : 1.5868229866027832
MAE --> all_data : 0.9087961316108704, vif_data: 0.9144780039787292, rfe_data : 0.9104907512664795
R2_Score --> all_data : 0.44986414909362793, vif_data: 0.44640934467315674, rfe_data : 0.44739341735839844


In [64]:
import matplotlib.pyplot as plt

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.facecolor'] = 'white'

shap_values = shap.TreeExplainer(model_lgb).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar", max_display=int(len(X_train.columns)))
plt.show()